In [74]:
from tree_sitter import Language, Parser

In [75]:
import torch
import numpy
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from transformers import RobertaTokenizer
from tqdm import tqdm, trange
import numpy as np

In [76]:
# Set the device      
device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [77]:
from model2 import Seq2Seq

In [78]:
trainned_model4 = 'D:\dilan_files\FYP_code\Pytorch_models\complete_model_dictv1.pt'
tokenizer = 'D:\dilan_files\FYP_code\Pytorch_models\complete_tokenizerv5.pt'

import torch
from transformers import RobertaTokenizer
model = torch.load(trainned_model4, map_location=torch.device('cpu'))
tokenizer = torch.load(tokenizer, map_location=torch.device('cpu'))

In [79]:
from transformers import RobertaConfig, RobertaModel
import torch.nn as nn

In [80]:
config_name = "microsoft/graphcodebert-base"
config = RobertaConfig.from_pretrained(config_name)
encoder = RobertaModel.from_pretrained(config_name, config=config)

decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)

model = Seq2Seq(encoder,decoder,config,beam_size=5,max_length=100,sos_id=0,eos_id=2)
model.load_state_dict(torch.load(trainned_model4, map_location=torch.device('cpu')))
model.eval()


Some weights of the model checkpoint at microsoft/graphcodebert-base were not used when initializing RobertaModel: ['lm_head.decoder.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to

Seq2Seq(
  (encoder): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [84]:
from DFG import (
    DFG_java,
    DFG_javascript
)
from utils import (
    remove_comments_and_docstrings,
    tree_to_token_index,
    index_to_code_token, 
    tree_to_variable_index
)

dfg_function={
    'java':DFG_java,
    'javascript':DFG_javascript
}

In [85]:
def extract_dataflow(code, parser,lang):
    #remove comments
    try:
        code=remove_comments_and_docstrings(code,lang)
    except:
        pass    
    #obtain dataflow
    # if lang=="php":
    #     code="<?php"+code+"?>"    
    try:
        tree = parser[0].parse(bytes(code,'utf8'))    
        root_node = tree.root_node  
        tokens_index=tree_to_token_index(root_node)     
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]  
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)  
        try:
            DFG,_=parser[1](root_node,index_to_code,{}) 
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg



In [86]:


language_path = './build_parser/my-languages.so'

In [87]:
#load parsers
parsers={}        
for lang in dfg_function:
    print("lang",lang)
    LANGUAGE = Language('./build_parser/java_js.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]    
    parsers[lang]= parser
    
print("Parsers :",parsers)

lang java
lang javascript
Parsers : {'java': [<tree_sitter.Parser object at 0x0000015A40E0EE90>, <function DFG_java at 0x00000159B6480790>], 'javascript': [<tree_sitter.Parser object at 0x0000015A14C979D0>, <function DFG_javascript at 0x00000159B75094C0>]}


In [88]:
# import javalang
java_code = """
ArrayList < Integer > al = new ArrayList < > ( ) ;
if (al.length > 4) {
  System.out.println("greater than 4")
}
"""

In [89]:
class Example(object):
    """A single training/test example."""
    def __init__(self,source, lang):
        self.source = source
        self.lang = lang


example = Example(source=java_code.strip(), lang='java')
examples = []
examples.append(example)

print("Parsers :",parsers)
code_tokens, dfg=extract_dataflow(example.source, parsers['java'], 'java')
code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]

Parsers : {'java': [<tree_sitter.Parser object at 0x0000015A40E0EE90>, <function DFG_java at 0x00000159B6480790>], 'javascript': [<tree_sitter.Parser object at 0x0000015A14C979D0>, <function DFG_javascript at 0x00000159B75094C0>]}


In [90]:
class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,                 
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.position_idx = position_idx
        self.dfg_to_code = dfg_to_code
        self.dfg_to_dfg = dfg_to_dfg
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask  


In [91]:
max_source_length = 320
max_target_length = 256
source_lang = "java"
target_lang = "javascript"

def convert_examples_to_features(examples, tokenizer, stage=None):
    features = []
    for example_index, example in enumerate(tqdm(examples,total=len(examples))):
        ##extract data flow
        # check for any error happened in this code
        # check if the data flow extraction is done for both the languages.
        code_tokens,dfg=extract_dataflow(example.source,
                                         parsers["java"],
                                         "java")
        code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
        # ORIGINAL TO CURRENT TOKEN POSITION
        ori2cur_pos={}
        ori2cur_pos[-1]=(0,0)
        for i in range(len(code_tokens)):
            ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))    

        flattened_code_tokens = []
        for x in code_tokens:
            for y in x:
                flattened_code_tokens.append(y)
        code_tokens = flattened_code_tokens
  
        
        #truncating
        code_tokens=code_tokens[:max_source_length-3][:512-3]
        
        # Adds the special tokens [CLS] and [SEP] to the beginning and end of the token 
        source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
        # convert the tokens to their corresponding token ids
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
        """
        Creates a position_idx sequence that assigns a unique position index to each token in
        the sequence. The '+1' offset is added to avoid the '0' position index which is reserved 
        for padding.

        position_idx = []
        for i in range(len(source_tokens)):
            position_idx.append(i + tokenizer.pad_token_id + 1)

        """
        position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(source_tokens))]
        dfg=dfg[:max_source_length-len(source_tokens)]
        """
        Concatenates the dfg edge labels with the source_tokens list and adds a 0 to the
        position_idx list for each dfg edge label.


        """
        source_tokens+=[x[0] for x in dfg]
        """
        Sets the token ids for the dfg edge labels to tokenizer.unk_token_id, indicating 
        that they are unknown tokens.
        """
        position_idx+=[0 for x in dfg]
        source_ids+=[tokenizer.unk_token_id for x in dfg]
        """
        Pads the position_idx and source_ids lists with tokenizer.pad_token_id to ensure 
        that they have the same length as max_source_length.
        """
        padding_length=max_source_length-len(source_ids)
        position_idx+=[tokenizer.pad_token_id]*padding_length
        source_ids+=[tokenizer.pad_token_id]*padding_length  

        """
        Creates a source_mask list that indicates which tokens in the sequence are valid 
        input tokens (1) and which ones are padding tokens (0)
        """
        source_mask = [1] * (len(source_tokens))
        source_mask+=[0]*padding_length       
        
        #reindex
        reverse_index={}
        for idx,x in enumerate(dfg):
            reverse_index[x[1]]=idx

        for idx,x in enumerate(dfg):
            dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)    

        
        
        dfg_to_dfg=[x[-1] for x in dfg]

        """
        creates a list dfg_to_code where each element corresponds to a tuple containing 
        the starting and ending positions of the code segment that corresponds to each node 
        in the dataflow graph (dfg). The positions are expressed in terms of the original source code.

        dfg_to_code = []
        for node in dfg:
            # Get the position of the node in the original source code
            position = ori2cur_pos[node[1]]
            # Add the position to the list
            dfg_to_code.append(position)

        
        """
        dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
        length=len([tokenizer.cls_token]) # length of the special token [CLS], always 1

        """ 
        for x in dfg_to_code:
            new_pos = (x[0] + length, x[1] + length) # Shift the start and end position by length of CLS token
            dfg_to_code.append(new_pos) # Add the new positions to the list
        """
        dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]        
      

        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:max_target_length-2]

        
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]            
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length   
   
        if example_index < 5:
            if stage=='train':
                print("*** Example ***")
                print("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                print("source_ids: {}".format(' '.join(map(str, source_ids))))
                print("source_mask: {}".format(' '.join(map(str, source_mask))))
                print("position_idx: {}".format(position_idx))
                print("dfg_to_code: {}".format(' '.join(map(str, dfg_to_code))))
                print("dfg_to_dfg: {}".format(' '.join(map(str, dfg_to_dfg))))
                
                print("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                print("target_ids: {}".format(' '.join(map(str, target_ids))))
                print("target_mask: {}".format(' '.join(map(str, target_mask))))
       
        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

In [92]:
class TextDataset(Dataset):
    def __init__(self, examples, max_source_length):
        self.examples = examples
        self.max_source_length = max_source_length
        
    def __len__(self):
        return len(self.examples)
    
    def __getitem__(self, item):
        #calculate graph-guided masked function
        attn_mask=np.zeros((self.max_source_length,self.max_source_length),dtype=np.bool)
        
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])

        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].source_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes         
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True  
                    
        return (torch.tensor(self.examples[item].source_ids),
                torch.tensor(self.examples[item].source_mask),
                torch.tensor(self.examples[item].position_idx),
                torch.tensor(attn_mask), 
                torch.tensor(self.examples[item].target_ids),
                torch.tensor(self.examples[item].target_mask),)

In [93]:
# self composed

# eval_examples = read_examples(file)
eval_features = convert_examples_to_features(examples, tokenizer,stage='test')
eval_data = TextDataset(eval_features, max_source_length) 

# Calculate bleu
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=8,num_workers=0)

100%|██████████| 1/1 [00:00<00:00, 195.68it/s]


In [94]:

#model.eval()

p=[]
translated_code = ""
for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
    batch = tuple(t.to(device) for t in batch)
    source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch                    
    with torch.no_grad():
        preds = model(source_ids,source_mask,position_idx,att_mask)
        for pred in preds:
            t=pred[0].cpu().numpy()
            t=list(t)
            print(t)
            if 0 in t:
                t=t[:t.index(0)]
            text = tokenizer.decode(t,clean_up_tokenization_spaces=False)
            translated_code = text
            p.append(text)

  0%|          | 0/1 [00:00<?, ?it/s]C:\Users\dilan\AppData\Local\Temp\ipykernel_24324\3594613766.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  attn_mask=np.zeros((self.max_source_length,self.max_source_length),dtype=np.bool)
  0%|          | 0/1 [00:00<?, ?it/s]


TypeError: type torch.cuda.LongTensor not available. Torch not compiled with CUDA enabled.